# Protocol Examples: Experimental Tests A, B, and C

**Level**: Advanced  
**Prerequisites**: Understanding of discrimination workflow

---

## Overview

This notebook implements three experimental protocols from **Paper Section 10.5**:

### Protocol A: Reset Test for History Dependence
Tests whether nominally perfect reset operations can eliminate memory effects. If spiral-time memory is intrinsic, residual history dependence should persist beyond experimental systematics.

### Protocol B: Process Tensor Reconstruction
Reconstructs multi-time quantum processes and tests whether they can be described by CP-divisible (Markovian) dynamics or require non-factorizing temporal correlations.

### Protocol C: Leggett-Garg Inequality under Memory Suppression
Tests multi-time correlations using Leggett-Garg inequalities with and without engineered memory suppression. Spiral-time predicts persistent violations even with memory suppression protocols.

**Goal**: Provide practical implementation examples for each protocol

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import sqrtm, expm
from scipy.stats import chi2_contingency
from itertools import product
from typing import Dict, List, Tuple, Optional, Callable
from dataclasses import dataclass, field
import warnings

np.random.seed(2026)
plt.style.use('seaborn-v0_8-whitegrid')

print("✓ Environment ready")
print("\n" + "="*70)
print("PROTOCOL EXAMPLES - EXPERIMENTAL TESTS A, B, C")
print("="*70)

---

## Protocol A: Reset Test for History Dependence

### Physical Intuition

**Question**: Can a "perfect" reset operation eliminate all memory?

**Standard QM**: Yes - reset to |0⟩ erases all history

**Spiral-Time**: No - memory sector χ persists through reset

### Test Strategy

1. Prepare system with different measurement histories
2. Apply "perfect" reset operation
3. Measure final state
4. Check if outcome probabilities depend on history

**Expected Result**:
- Markovian: No history dependence after reset
- Spiral-Time: Persistent history dependence (encoded in χ)

In [ ]:
@dataclass
class ResetTestConfig:
    """Configuration for reset test protocol."""
    n_cycles: int = 100
    n_histories: int = 10
    history_length: int = 5
    reset_fidelity: float = 0.999
    measurement_basis: str = "computational"
    memory_strength: float = 0.01
    memory_decay_time: float = 10.0
    confidence_level: float = 0.95
    min_effect_size: float = 0.001


class QuantumState:
    """Represents a qubit state with optional memory sector."""
    
    def __init__(self, rho: np.ndarray, chi: float = 0.0):
        self.rho = rho / np.trace(rho)
        self.chi = chi
        self._validate()
    
    def _validate(self):
        if not np.allclose(self.rho, self.rho.conj().T):
            raise ValueError("Density matrix must be Hermitian")
        if not np.isclose(np.trace(self.rho), 1.0):
            raise ValueError("Density matrix must have unit trace")
        eigenvalues = np.linalg.eigvalsh(self.rho)
        if np.any(eigenvalues < -1e-10):
            raise ValueError("Density matrix must be positive semidefinite")
    
    def copy(self):
        return QuantumState(self.rho.copy(), self.chi)


class ResetOperation:
    """Models a reset operation with configurable fidelity."""
    
    def __init__(self, fidelity: float = 1.0, target_state: Optional[np.ndarray] = None):
        self.fidelity = fidelity
        if target_state is None:
            self.target_state = np.array([[1.0, 0.0], [0.0, 0.0]], dtype=complex)
        else:
            self.target_state = target_state
    
    def apply(self, state: QuantumState, preserve_memory: bool = False):
        """Apply reset: rho -> fidelity * |0⟩⟨0| + (1-fidelity) * rho"""
        reset_rho = (self.fidelity * self.target_state + 
                    (1 - self.fidelity) * state.rho)
        
        if preserve_memory:
            reset_chi = state.chi  # Spiral-time: chi persists
        else:
            reset_chi = 0.0  # Markovian: chi also reset
        
        return QuantumState(reset_rho, reset_chi)


class MeasurementOperation:
    """Projective measurement in specified basis."""
    
    def __init__(self, basis: str = "computational"):
        self.basis = basis
        if basis == "computational":
            self.projectors = [
                np.array([[1, 0], [0, 0]], dtype=complex),
                np.array([[0, 0], [0, 1]], dtype=complex),
            ]
        elif basis == "hadamard":
            self.projectors = [
                np.array([[0.5, 0.5], [0.5, 0.5]], dtype=complex),
                np.array([[0.5, -0.5], [-0.5, 0.5]], dtype=complex),
            ]
    
    def measure(self, state: QuantumState, update_memory: bool = True,
                memory_strength: float = 0.01):
        """Perform measurement and update memory sector."""
        probs = [np.real(np.trace(P @ state.rho)) for P in self.projectors]
        probs = np.array(probs) / sum(probs)
        
        outcome = np.random.choice(len(probs), p=probs)
        
        P = self.projectors[outcome]
        post_rho = P @ state.rho @ P
        post_rho = post_rho / np.trace(post_rho)
        
        if update_memory:
            delta_chi = memory_strength * (2 * outcome - 1)
            post_chi = state.chi + delta_chi
        else:
            post_chi = state.chi
        
        return outcome, QuantumState(post_rho, post_chi)


def run_reset_test(config: ResetTestConfig, spiral_time_mode: bool = True):
    """Run Protocol A: Reset test for history dependence."""
    print(f"\nRunning Protocol A: Reset Test")
    print(f"Mode: {'Spiral-Time' if spiral_time_mode else 'Markovian'}")
    
    reset_op = ResetOperation(fidelity=config.reset_fidelity)
    measure_op = MeasurementOperation(basis=config.measurement_basis)
    
    all_outcomes = []
    all_histories = []
    all_chi_values = []
    
    measurement_sequences = [
        tuple(np.random.randint(0, 2, config.history_length))
        for _ in range(config.n_histories)
    ]
    
    for cycle in range(config.n_cycles):
        history = measurement_sequences[cycle % config.n_histories]
        state = QuantumState(np.array([[1, 0], [0, 0]], dtype=complex), chi=0.0)
        
        # Apply measurement history
        for target_outcome in history:
            if target_outcome == 1:
                state.rho = np.array([[0, 0], [0, 1]], dtype=complex)
            outcome, state = measure_op.measure(
                state, update_memory=spiral_time_mode,
                memory_strength=config.memory_strength
            )
        
        # Apply reset
        state = reset_op.apply(state, preserve_memory=spiral_time_mode)
        
        # Final measurement
        outcome, state = measure_op.measure(state, update_memory=False)
        
        all_outcomes.append(outcome)
        all_histories.append(history)
        all_chi_values.append(state.chi)
    
    # Analyze history dependence
    unique_histories = sorted(set(all_histories))
    unique_outcomes = sorted(set(all_outcomes))
    
    contingency = np.zeros((len(unique_histories), len(unique_outcomes)))
    for i, hist in enumerate(unique_histories):
        for j, outcome in enumerate(unique_outcomes):
            mask = [h == hist and o == outcome 
                   for h, o in zip(all_histories, all_outcomes)]
            contingency[i, j] = sum(mask)
    
    chi2, p_value, dof, expected = chi2_contingency(contingency)
    n = contingency.sum()
    cramers_v = np.sqrt(chi2 / (n * (min(contingency.shape) - 1)))
    
    memory_persistence = np.std(all_chi_values) if spiral_time_mode else 0.0
    
    print(f"  Chi-squared: {chi2:.4f}")
    print(f"  p-value: {p_value:.4e}")
    print(f"  Cramér's V: {cramers_v:.4f}")
    print(f"  Memory persistence: {memory_persistence:.4f}")
    
    significant = p_value < (1 - config.confidence_level)
    
    if spiral_time_mode and significant:
        print(f"  ✓ History dependence persists after reset (spiral-time)")
    elif not spiral_time_mode and not significant:
        print(f"  ✓ No history dependence after reset (Markovian)")
    
    return {
        'chi_squared': chi2,
        'p_value': p_value,
        'cramers_v': cramers_v,
        'memory_persistence': memory_persistence,
        'significant': significant
    }


# Run Protocol A
print("\n" + "="*70)
print("PROTOCOL A: RESET TEST")
print("="*70)

config_a = ResetTestConfig(
    n_cycles=500,
    n_histories=8,
    history_length=4,
    reset_fidelity=0.999,
    memory_strength=0.02
)

results_a_spiral = run_reset_test(config_a, spiral_time_mode=True)
results_a_markov = run_reset_test(config_a, spiral_time_mode=False)

print("\n📊 Protocol A Summary:")
print(f"  Spiral-time shows history dependence: {results_a_spiral['significant']}")
print(f"  Markovian shows history dependence: {results_a_markov['significant']}")

---

## Protocol B: Process Tensor Reconstruction

### Physical Intuition

**Question**: Can we describe evolution as a sequence of independent operations?

**Markovian Process**: E(t₃,t₀) = E(t₃,t₂) ∘ E(t₂,t₁) ∘ E(t₁,t₀)

**Non-Markovian**: Process tensor has non-factorizing correlations

### Test Strategy

1. Apply all possible operation sequences at multiple times
2. Reconstruct full process tensor from outcomes
3. Test CP-divisibility: Can we factorize as Markov chain?
4. Check eigenvalue monotonicity (CP-divisibility signature)

**Expected Result**:
- Markovian: CP-divisible (factorizes)
- Spiral-Time: Non-CP-divisible (intrinsic memory)

In [ ]:
@dataclass
class ProcessTensorConfig:
    """Configuration for process tensor tomography."""
    n_time_steps: int = 3
    n_measurements: int = 1000
    time_step_duration: float = 1.0
    memory_strength: float = 0.01
    memory_kernel_type: str = "exponential"
    memory_decay_rate: float = 0.5


class PauliOperators:
    """Pauli operator basis."""
    I = np.array([[1, 0], [0, 1]], dtype=complex)
    X = np.array([[0, 1], [1, 0]], dtype=complex)
    Y = np.array([[0, -1j], [1j, 0]], dtype=complex)
    Z = np.array([[1, 0], [0, -1]], dtype=complex)
    basis = [I, X, Y, Z]
    labels = ['I', 'X', 'Y', 'Z']


class MemoryKernel:
    """Memory kernel K(t-τ)."""
    
    def __init__(self, kernel_type: str = "exponential", 
                 strength: float = 0.01, decay_rate: float = 0.5):
        self.kernel_type = kernel_type
        self.strength = strength
        self.decay_rate = decay_rate
    
    def __call__(self, t: float, tau: float) -> float:
        dt = t - tau
        if dt < 0:
            return 0.0
        
        if self.kernel_type == "exponential":
            return self.strength * np.exp(-self.decay_rate * dt)
        elif self.kernel_type == "power_law":
            return self.strength / (1 + self.decay_rate * dt)
        else:
            raise ValueError(f"Unknown kernel type: {self.kernel_type}")


class NonMarkovianEvolution:
    """Non-Markovian evolution with memory."""
    
    def __init__(self, kernel: MemoryKernel, dt: float = 0.01):
        self.kernel = kernel
        self.dt = dt
        self.history = []
    
    def evolve(self, rho: np.ndarray, t_final: float) -> np.ndarray:
        """Evolve with memory: ρ̇ = L[ρ] + ∫K(t-τ)ρ(τ)dτ"""
        def lindblad(rho_t):
            sigma_z = PauliOperators.Z
            dephasing_rate = 0.1
            return dephasing_rate * (sigma_z @ rho_t @ sigma_z - rho_t)
        
        self.history = [(0.0, rho.copy())]
        t = 0.0
        rho_current = rho.copy()
        
        while t < t_final:
            drho_local = lindblad(rho_current)
            memory_contribution = np.zeros_like(rho_current)
            
            for t_past, rho_past in self.history:
                if t_past <= t:
                    kernel_val = self.kernel(t, t_past)
                    memory_contribution += kernel_val * rho_past * self.dt
            
            drho_total = drho_local + memory_contribution
            rho_current = rho_current + drho_total * self.dt
            
            # Ensure physicality
            rho_current = (rho_current + rho_current.conj().T) / 2
            rho_current = rho_current / np.trace(rho_current)
            eigvals, eigvecs = np.linalg.eigh(rho_current)
            eigvals = np.maximum(eigvals, 0)
            rho_current = eigvecs @ np.diag(eigvals) @ eigvecs.conj().T
            rho_current = rho_current / np.trace(rho_current)
            
            t += self.dt
            self.history.append((t, rho_current.copy()))
        
        return rho_current


class ProcessTensor:
    """Multi-time quantum process."""
    
    def __init__(self, n_steps: int):
        self.n_steps = n_steps
        self.tensor_data = {}
    
    def add_measurement(self, operations: Tuple[int, ...], 
                       final_state: np.ndarray, weight: float = 1.0):
        if operations not in self.tensor_data:
            self.tensor_data[operations] = []
        self.tensor_data[operations].append((final_state, weight))
    
    def predict_outcome(self, operations: Tuple[int, ...]):
        if operations not in self.tensor_data:
            return None
        states, weights = zip(*self.tensor_data[operations])
        total_weight = sum(weights)
        return sum(w * s for w, s in zip(weights, states)) / total_weight


def test_cp_divisibility(process_tensor: ProcessTensor, time_steps: List[int]):
    """Test if process is CP-divisible."""
    violations = []
    
    for t in time_steps[1:]:
        for s in range(1, t):
            identity_seq = tuple([0] * t)
            state_0 = process_tensor.predict_outcome(tuple([0] * 0))
            state_s = process_tensor.predict_outcome(tuple([0] * s))
            state_t = process_tensor.predict_outcome(identity_seq)
            
            if state_s is not None and state_t is not None:
                eig_s = np.linalg.eigvalsh(state_s)
                eig_t = np.linalg.eigvalsh(state_t)
                
                # Non-monotonic eigenvalues indicate non-CP-divisibility
                if not np.all(eig_t <= eig_s + 1e-6):
                    violations.append({'time': t, 'intermediate': s})
    
    return {
        'is_cp_divisible': len(violations) == 0,
        'violations': violations,
        'n_violations': len(violations)
    }


def reconstruct_process_tensor(config: ProcessTensorConfig, use_memory: bool = True):
    """Reconstruct process tensor from measurements."""
    print(f"\nReconstructing process tensor: {config.n_time_steps} steps")
    print(f"Memory: {'Enabled' if use_memory else 'Disabled'}")
    
    if use_memory:
        kernel = MemoryKernel(
            kernel_type=config.memory_kernel_type,
            strength=config.memory_strength,
            decay_rate=config.memory_decay_rate
        )
        evolver = NonMarkovianEvolution(kernel, dt=0.01)
    
    process_tensor = ProcessTensor(config.n_time_steps)
    rho_init = np.array([[1, 0], [0, 0]], dtype=complex)
    n_operations = 4
    
    for ops_sequence in product(range(n_operations), repeat=config.n_time_steps):
        rho = rho_init.copy()
        
        for step, op_idx in enumerate(ops_sequence):
            op = PauliOperators.basis[op_idx]
            rho = op @ rho @ op.conj().T
            rho = rho / np.trace(rho)
            
            if use_memory:
                rho = evolver.evolve(rho, config.time_step_duration)
            else:
                dephasing = 0.1 * config.time_step_duration
                rho = (1 - dephasing) * rho + dephasing * np.diag(np.diag(rho))
        
        process_tensor.add_measurement(ops_sequence, rho)
    
    return process_tensor


# Run Protocol B
print("\n" + "="*70)
print("PROTOCOL B: PROCESS TENSOR RECONSTRUCTION")
print("="*70)

config_b = ProcessTensorConfig(
    n_time_steps=3,
    memory_strength=0.05,
    memory_kernel_type="exponential",
    memory_decay_rate=0.3
)

tensor_memory = reconstruct_process_tensor(config_b, use_memory=True)
tensor_markov = reconstruct_process_tensor(config_b, use_memory=False)

time_steps = list(range(1, config_b.n_time_steps + 1))
cp_test_memory = test_cp_divisibility(tensor_memory, time_steps)
cp_test_markov = test_cp_divisibility(tensor_markov, time_steps)

print("\n📊 Protocol B Summary:")
print(f"  Spiral-time CP-divisible: {cp_test_memory['is_cp_divisible']}")
print(f"  Violations: {cp_test_memory['n_violations']}")
print(f"  Markovian CP-divisible: {cp_test_markov['is_cp_divisible']}")
print(f"  Violations: {cp_test_markov['n_violations']}")

---

## Protocol C: Leggett-Garg Inequality under Memory Suppression

### Physical Intuition

**Leggett-Garg Inequality**: Classical bound on temporal correlations

K₃ = C(t₁,t₂) + C(t₂,t₃) - C(t₁,t₃) ≤ 1 (classical)

**Question**: Can memory suppression eliminate violations?

### Test Strategy

1. Measure K₃ without memory suppression → expect violation
2. Apply memory suppression (dynamical decoupling, frequent reset)
3. Measure K₃ again
4. Compare violation magnitude

**Expected Result**:
- Environmental memory: Suppression eliminates violation
- Intrinsic memory: Violation persists despite suppression

In [ ]:
@dataclass
class LeggettGargConfig:
    """Configuration for Leggett-Garg protocol."""
    n_measurements: int = 10000
    n_time_points: int = 3
    time_intervals: List[float] = field(default_factory=list)
    memory_strength: float = 0.02
    memory_coherence_time: float = 5.0
    suppression_enabled: bool = False
    suppression_strength: float = 0.5
    suppression_method: str = "dynamical_decoupling"
    
    def __post_init__(self):
        if not self.time_intervals:
            self.time_intervals = [1.0] * (self.n_time_points - 1)


class QuantumEvolution:
    """Simulates qubit evolution with optional memory effects."""
    
    def __init__(self, memory_strength: float, coherence_time: float):
        self.memory_strength = memory_strength
        self.coherence_time = coherence_time
        self.chi = 0.0
    
    def evolve_with_memory(self, rho: np.ndarray, dt: float, 
                          suppression: float = 0.0) -> np.ndarray:
        omega = 1.0
        H = 0.5 * omega * np.array([[1, 0], [0, -1]], dtype=complex)
        U = np.array([
            [np.cos(omega * dt / 2) - 1j * np.sin(omega * dt / 2), 0],
            [0, np.cos(omega * dt / 2) + 1j * np.sin(omega * dt / 2)]
        ])
        
        rho_evolved = U @ rho @ U.conj().T
        
        effective_memory = self.memory_strength * (1 - suppression)
        memory_factor = np.exp(-effective_memory * dt / self.coherence_time)
        
        rho_evolved[0, 1] *= memory_factor
        rho_evolved[1, 0] *= memory_factor
        
        self.chi += effective_memory * np.real(rho_evolved[0, 0] - rho_evolved[1, 1])
        
        return rho_evolved
    
    def reset_memory(self, strength: float = 1.0):
        self.chi *= (1 - strength)


class LeggettGargMeasurement:
    """Implements projective measurements for Leggett-Garg tests."""
    
    def __init__(self, observable: str = "Z"):
        self.observable = observable
        
        if observable == "Z":
            self.operator = np.array([[1, 0], [0, -1]], dtype=complex)
            self.projectors = [
                np.array([[1, 0], [0, 0]], dtype=complex),
                np.array([[0, 0], [0, 1]], dtype=complex),
            ]
            self.outcomes = [+1, -1]
        elif observable == "X":
            self.operator = np.array([[0, 1], [1, 0]], dtype=complex)
            self.projectors = [
                0.5 * np.array([[1, 1], [1, 1]], dtype=complex),
                0.5 * np.array([[1, -1], [-1, 1]], dtype=complex),
            ]
            self.outcomes = [+1, -1]
        else:
            raise ValueError(f"Unsupported observable: {observable}")
    
    def measure(self, rho: np.ndarray, collapse: bool = True) -> Tuple[float, np.ndarray]:
        expectation = np.real(np.trace(self.operator @ rho))
        
        if collapse:
            probs = [np.real(np.trace(P @ rho)) for P in self.projectors]
            probs = np.array(probs) / sum(probs)
            
            outcome_idx = np.random.choice(len(probs), p=probs)
            outcome = self.outcomes[outcome_idx]
            
            P = self.projectors[outcome_idx]
            rho_collapsed = P @ rho @ P
            rho_collapsed = rho_collapsed / np.trace(rho_collapsed)
            
            return outcome, rho_collapsed
        else:
            return expectation, rho


class LeggettGargInequality:
    """Computes and tests Leggett-Garg inequalities."""
    
    @staticmethod
    def compute_K3(correlators: Dict[Tuple[int, int], float]) -> float:
        C_12 = correlators.get((0, 1), 0)
        C_23 = correlators.get((1, 2), 0)
        C_13 = correlators.get((0, 2), 0)
        return C_12 + C_23 - C_13
    
    @staticmethod
    def test_violation(K: float, n_times: int = 3) -> Dict:
        if n_times == 3:
            classical_max = 1.0
            classical_min = -3.0
        elif n_times == 4:
            classical_max = 2.0
            classical_min = -4.0
        else:
            raise ValueError("Only 3 or 4 time points supported")
        
        violates_upper = K > classical_max
        violates_lower = K < classical_min
        violation_magnitude = max(K - classical_max, classical_min - K, 0)
        
        return {
            'K': K,
            'classical_max': classical_max,
            'violates': violates_upper or violates_lower,
            'violation_magnitude': violation_magnitude
        }


def run_leggett_garg_experiment(config: LeggettGargConfig, invasive: bool = True) -> Dict:
    evolver = QuantumEvolution(config.memory_strength, config.memory_coherence_time)
    measurement = LeggettGargMeasurement(observable="Z")
    
    measurement_results = {i: [] for i in range(config.n_time_points)}
    
    for run in range(config.n_measurements):
        rho = 0.5 * np.array([[1, 1], [1, 1]], dtype=complex)
        evolver.reset_memory()
        
        for t_idx in range(config.n_time_points):
            if t_idx > 0:
                dt = config.time_intervals[t_idx - 1]
                
                if config.suppression_enabled:
                    if config.suppression_method == "dynamical_decoupling":
                        rho = evolver.evolve_with_memory(
                            rho, dt, suppression=config.suppression_strength
                        )
                    elif config.suppression_method == "frequent_reset":
                        rho = evolver.evolve_with_memory(rho, dt, suppression=0.0)
                        evolver.reset_memory(strength=config.suppression_strength)
                else:
                    rho = evolver.evolve_with_memory(rho, dt, suppression=0.0)
            
            outcome, rho = measurement.measure(rho, collapse=invasive)
            measurement_results[t_idx].append(outcome)
    
    correlators = {}
    for i in range(config.n_time_points):
        for j in range(i + 1, config.n_time_points):
            correlators[(i, j)] = np.mean([
                measurement_results[i][k] * measurement_results[j][k]
                for k in range(config.n_measurements)
            ])
    
    K = LeggettGargInequality.compute_K3(correlators) if config.n_time_points == 3 else 0.0
    test_result = LeggettGargInequality.test_violation(K, config.n_time_points)
    
    return {'correlators': correlators, 'K': K, 'test_result': test_result}


# Run Protocol C
print("\n" + "="*70)
print("PROTOCOL C: LEGGETT-GARG INEQUALITY")
print("="*70)

config_c = LeggettGargConfig(
    n_measurements=5000,
    n_time_points=3,
    time_intervals=[1.0, 1.0],
    memory_strength=0.03,
    suppression_strength=0.7
)

print("\nWithout memory suppression:")
config_c.suppression_enabled = False
results_c_no_supp = run_leggett_garg_experiment(config_c)
print(f"  K = {results_c_no_supp['K']:.4f}")
print(f"  Violation: {results_c_no_supp['test_result']['violates']}")

print("\nWith memory suppression:")
config_c.suppression_enabled = True
results_c_with_supp = run_leggett_garg_experiment(config_c)
print(f"  K = {results_c_with_supp['K']:.4f}")
print(f"  Violation: {results_c_with_supp['test_result']['violates']}")

delta_K = results_c_no_supp['K'] - results_c_with_supp['K']
print(f"\nChange in K: ΔK = {delta_K:.4f}")

if results_c_with_supp['test_result']['violates']:
    print("✓ SPIRAL-TIME: Violation persists despite suppression")
else:
    print("✗ Violation eliminated (environmental memory)")

---

## Visualization: Protocol Results

Let's visualize the key results from all three protocols.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Protocol A: History dependence
ax = axes[0]
protocols = ['Markovian', 'Spiral-Time']
cramers_v = [results_a_markov['cramers_v'], results_a_spiral['cramers_v']]
colors = ['#3498db', '#e74c3c']
bars = ax.bar(protocols, cramers_v, color=colors, alpha=0.7, edgecolor='black')
ax.axhline(y=0.1, color='gray', linestyle='--', label='Threshold')
ax.set_ylabel("Cramér's V (Effect Size)")
ax.set_title('Protocol A: Reset Test')
ax.set_ylim([0, max(cramers_v) * 1.2])
ax.legend()
ax.grid(axis='y', alpha=0.3)

# Protocol B: CP-divisibility violations
ax = axes[1]
protocols = ['Markovian', 'Spiral-Time']
violations = [cp_test_markov['n_violations'], cp_test_memory['n_violations']]
bars = ax.bar(protocols, violations, color=colors, alpha=0.7, edgecolor='black')
ax.set_ylabel('Number of CP Violations')
ax.set_title('Protocol B: Process Tensor')
ax.set_ylim([0, max(violations) * 1.2 if max(violations) > 0 else 1])
ax.grid(axis='y', alpha=0.3)

# Protocol C: Leggett-Garg K values
ax = axes[2]
conditions = ['No Suppression', 'With Suppression']
K_values = [results_c_no_supp['K'], results_c_with_supp['K']]
bars = ax.bar(conditions, K_values, color=['#e74c3c', '#f39c12'], alpha=0.7, edgecolor='black')
ax.axhline(y=1.0, color='black', linestyle='-', linewidth=2, label='Classical Bound')
ax.set_ylabel('K₃ Parameter')
ax.set_title('Protocol C: Leggett-Garg')
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig('protocol_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Visualization complete")

---

## Summary and Conclusions

### Protocol Comparison

| Protocol | Test | Markovian | Spiral-Time |
|----------|------|-----------|-------------|
| A | Reset | No history dependence | Persistent history dependence |
| B | Process tensor | CP-divisible | Non-CP-divisible |
| C | Leggett-Garg | Suppression works | Violation persists |

### Key Insights

1. **Protocol A** tests whether memory can be erased by resets
2. **Protocol B** tests whether evolution factorizes as Markov chain
3. **Protocol C** tests whether memory is environmental vs intrinsic

All three protocols distinguish between:
- **Environmental memory**: Can be suppressed
- **Intrinsic temporal memory**: Cannot be eliminated

### What We've Demonstrated

These three protocols provide complementary evidence for distinguishing spiral-time from standard quantum mechanics:

1. **Protocol A (Reset Test)**
   - Shows that memory persists through nominally perfect reset operations
   - Quantifies history dependence using statistical tests
   - Key signature: χ sector survives reset in spiral-time

2. **Protocol B (Process Tensor)**
   - Reconstructs multi-time quantum processes
   - Tests CP-divisibility (Markov property)
   - Key signature: Non-factorizing temporal correlations

3. **Protocol C (Leggett-Garg)**
   - Tests temporal correlations under memory suppression
   - Distinguishes environmental from intrinsic memory
   - Key signature: Violations persist despite suppression

### Experimental Implementation

These protocols can be implemented on current quantum platforms:
- Superconducting qubits
- Trapped ions
- Photonic systems
- NV centers

### Next Steps

1. Optimize parameters for specific experimental platforms
2. Add error mitigation and calibration procedures
3. Perform statistical power analysis
4. Design blind testing protocols

---

**End of Protocol Examples Notebook**